In [2]:
!pip install Flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 1.6 MB/s eta 0:00:00 0:00:01


In [3]:
import json
from flask import Flask, jsonify, request


In [9]:
app = Flask(__name__)

@app.route('/voice/detection', methods=['GET'])
def get_employees():
     return jsonify(employees)

@app.route('/employees/<int:id>', methods=['GET'])
def get_employee_by_id(id: int):
     employee = get_employee(id)
     if employee is None:
       return jsonify({ 'error': 'Employee does not exist'}), 404
     return jsonify(employee)

def get_employee(id):
     return next((e for e in employees if e['id'] == id), None)

def employee_is_valid(employee):
     for key in employee.keys():
       if key != 'name':
        return False
     return True

@app.route('/employees', methods=['POST'])
def create_employee():
     global nextEmployeeId
     employee = json.loads(request.data)
     if not employee_is_valid(employee):
       return jsonify({ 'error': 'Invalid employee properties.' }), 400

     employee['id'] = nextEmployeeId
     nextEmployeeId += 1
     employees.append(employee)

     return '', 201, { 'location': f'/employees/{employee["id"]}' }

@app.route('/employees/<int:id>', methods=['PUT'])
def update_employee(id: int):
     employee = get_employee(id)
     if employee is None:
       return jsonify({ 'error': 'Employee does not exist.' }), 404

     updated_employee = json.loads(request.data)
     if not employee_is_valid(updated_employee):
       return jsonify({ 'error': 'Invalid employee properties.' }), 400

     employee.update(updated_employee)

     return jsonify(employee)

@app.route('/employees/<int:id>', methods=['DELETE'])
def delete_employee(id: int):
     global employees
     employee = get_employee(id)
     if employee is None:
       return jsonify({ 'error': 'Employee does not exist.' }), 404

     employees = [e for e in employees if e['id'] != id]
     return jsonify(employee), 200

if __name__ == '__main__':
   app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [11]:
from flask import Flask, render_template, request, jsonify
import librosa
import pickle
import numpy as np

app = Flask(__name__)
model_pkl_file = "Xg_Boost_model.pkl" 

def detect_fake(file):
    sound_signal, sample_rate = librosa.load(file, res_type="kaiser_fast")
    mfcc_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfcc_features.T, axis=0)
    mfccs_features_scaled = mfccs_features_scaled.reshape(1, -1)
    result_array = model.predict(mfccs_features_scaled)
    print(result_array)
    result_classes = ["FAKE", "REAL"]
    result = np.argmax(result_array[0])
    print("Result:", result_classes[result])
	
@app.route('/upload', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
      f = request.files['file']
      with open(model_pkl_file, 'rb') as file:  
          model = pickle.load(file)
    sound_signal, sample_rate = librosa.load(f, res_type="kaiser_fast")
    mfcc_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)
    mfccs_features_scaled = np.mean(mfcc_features.T, axis=0)
    mfccs_features_scaled = mfccs_features_scaled.reshape(1, -1)
    result_array = model.predict(mfccs_features_scaled)
    print(result_array)
    result_classes = ["FAKE", "REAL"]
    result = np.argmax(result_array[0])
    print("Result:", result_classes[result])

    return result_classes[result] 
@app.route('/uploader', methods = ['GET', 'POST'])
def uploaded_file():
    print("inside")
    return "Ok"
		
if __name__ == '__main__':
   app.run(debug = True,port=4444)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:4444
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(sel

SystemExit: 1